### Traffic Trauma Puzzle ML solving

install all libraries

In [ ]:
!pip install opencv-python-headless matplotlib pandas

Then import all libraries

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

Load all images and define ROIs and then extract the colors and then the dominant color form each block to avoid input of the background color

In [ ]:
def get_dominant_color(colors):
    # Filter out the gray color since it it the background
    filtered_colors = [color for color in colors if not np.array_equal(color, [115, 115, 115])]
    
    if filtered_colors:
        # Count the occurrence of each color
        color_counts = Counter(tuple(color) for color in filtered_colors)
        # Return the most common color
        return np.array(color_counts.most_common(1)[0][0])
    else:
        return None

# save the colors in a list for the tiles
dominant_colors = []

# Load all tile images and mark the ROIs for visibility
tile_images = []
for i in range(1, 17):
    image_path = f"data/images/tile{i}.jpg"
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Failed to load image {image_path}")
    else:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        tile_images.append(img_rgb)  # add the image to the list
        
        height, width, _ = img_rgb.shape
        
        # Define the region of interest (ROI) for the top block
        # define the approximate size of the rectangle blocks representing cars
        block_height = int(height * 0.1)
        block_width = int(width * 0.3)
        
        # Top block
        top_left_x = int(width / 2 - block_width / 2)
        top_left_y = int(height * 0.05)
        bottom_right_x = top_left_x + block_width
        bottom_right_y = top_left_y + block_height
        cv2.rectangle(img_rgb, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 0, 255), 2)
        
        # Bottom block
        bottom_left_x = int(width / 2 - block_width / 2)
        bottom_left_y = int(height * 0.85)
        bottom_right_x = bottom_left_x + block_width
        bottom_right_y = bottom_left_y + block_height
        cv2.rectangle(img_rgb, (bottom_left_x, bottom_left_y), (bottom_right_x, bottom_right_y), (0, 0, 255), 2)
        
        # Left block
        block_height_left_right = int(height * 0.3)
        block_width_left_right = int(width * 0.1)
        left_top_x = int(width * 0.05)
        left_top_y = int(height / 2 - block_height_left_right / 2)
        right_bottom_x = left_top_x + block_width_left_right
        right_bottom_y = left_top_y + block_height_left_right
        cv2.rectangle(img_rgb, (left_top_x, left_top_y), (right_bottom_x, right_bottom_y), (0, 0, 255), 2)
        
        # Right block
        right_top_x = int(width * 0.85)
        right_top_y = int(height / 2 - block_height_left_right / 2)
        right_bottom_x = right_top_x + block_width_left_right
        right_bottom_y = right_top_y + block_height_left_right
        cv2.rectangle(img_rgb, (right_top_x, right_top_y), (right_bottom_x, right_bottom_y), (0, 0, 255), 2)
        
        # get color data from the blocks
        top_block_colors = img_rgb[top_left_y:bottom_right_y, top_left_x:bottom_right_x].reshape(-1, 3)
        bottom_block_colors = img_rgb[bottom_left_y:bottom_right_y, bottom_left_x:bottom_right_x].reshape(-1, 3)
        left_block_colors = img_rgb[left_top_y:right_bottom_y, left_top_x:right_bottom_x].reshape(-1, 3)
        right_block_colors = img_rgb[right_top_y:right_bottom_y, right_top_x:right_bottom_x].reshape(-1, 3)

        # Get the dominant colors for each block
        dominant_top_color = get_dominant_color(top_block_colors)
        dominant_bottom_color = get_dominant_color(bottom_block_colors)
        dominant_left_color = get_dominant_color(left_block_colors)
        dominant_right_color = get_dominant_color(right_block_colors)

        # Store the dominant colors for the tile
        dominant_colors.append({
            'Tile': f'Tile {i}',
            'Top Color': dominant_top_color,
            'Bottom Color': dominant_bottom_color,
            'Left Color': dominant_left_color,
            'Right Color': dominant_right_color
        })

# Display marked images in a 4x4 grid
plt.figure(figsize=(10, 10))  # Set the figure size
for i in range(16):
    plt.subplot(4, 4, i+1)  # Create a subplot for each image
    plt.imshow(tile_images[i])
    plt.title(f"Tile {i+1}")
    plt.axis('off')  # Hide axis

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()  # Show all images

Output the RGB values

In [ ]:
# Create a DataFrame to store the dominant colors
df = pd.DataFrame(dominant_colors)

# Save the DataFrame to a CSV file
df.to_csv('dominant_colors.csv', index=False)

# Display the DataFrame
df

Map RGB values to human readable color names

In [ ]:
# Function to map RGB values to color names
def rgb_to_color_name(rgb):
    color_mapping = {
        (0, 0, 0): "Black",
        (255, 255, 255): "White",
        (255, 255, 0): "Yellow",
        (255, 104, 31): "Orange",
        (0, 128, 1): "Green",
        (254, 0, 0): "Red",
        (175, 44, 172): "Violet",
        (75, 0, 129): "Indigo"
    }
    
    # Convert the string representation of RGB array into a tuple
    rgb = rgb.replace("[", "").replace("]", "").split()
    rgb_tuple = tuple(map(int, rgb))
    
    return color_mapping.get(rgb_tuple, "Unknown")

# Create a new list to store the mapped data
dominant_colors_named = []

# Convert the RGB values to human-readable names
for index, row in df.iterrows():
    dominant_colors_named.append({
        'Tile': row['Tile'],
        'Top Color': rgb_to_color_name(str(row['Top Color'])),
        'Bottom Color': rgb_to_color_name(str(row['Bottom Color'])),
        'Left Color': rgb_to_color_name(str(row['Left Color'])),
        'Right Color': rgb_to_color_name(str(row['Right Color']))
    })

# Create a DataFrame with color names
df_named = pd.DataFrame(dominant_colors_named)

# Save the new DataFrame to a CSV file
df_named.to_csv('dominant_colors_named.csv', index=False)

# Display the DataFrame with color names
df_named